In [1]:
import pydantic
print(pydantic.__version__)

2.10.2


Without the use of Pydantics

In [3]:
# If Pydantic is not used, custom type checking is necessary to check data type of each variable passed 
# into a function
class User:
    def __init__(self, id: int, name = 'Jane Doe'):
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name__}')
        
        if not isinstance(name, str):
            raise TypeError(f'Expected name to be an str, got {type(name).__name__}')
        
        self.id = id
        self.name = name
        print(id)
        print(name)
    
try:
    user = User(id= 123)
except TypeError as e:
    print(e)

123
Jane Doe


In [11]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = "Jane Doe"

#No error
user1 = User(id = "123")
print(user1)

#No error
user2 = User(id = 123)
print(user2)    

#Gives error
# user3 = User(id = "dsa")
# print(user3)

id=123 name='Jane Doe'
id=123 name='Jane Doe'


In [13]:
print(user.model_fields_set)
user = User(id = '123', name = 'Mokshin Vij')
print(user.model_fields_set)

{'id'}
{'id', 'name'}


In [14]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 123, 'name': 'Mokshin Vij'}
{"id":123,"name":"Mokshin Vij"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Jane Doe', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


Nested Models

In [18]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None

class Restuarant(BaseModel):
    name: str
    location: str
    foods: List[Food]
    
food1 = Food(name = "1st dish", price = 1, ingredients = ["123"])
food2 = Food(name = "2nd Dish", price = 2.85)

restuarant1 = Restuarant(name = "Dhaba", location = "Punjab", foods = [food1] )
restuarant2 = Restuarant(name = "Rasoi", location = "Rajasthan", foods = [food1, food2])

print(restuarant1)
print(restuarant2)
print(restuarant2.model_dump())

name='Dhaba' location='Punjab' foods=[Food(name='1st dish', price=1.0, ingredients=['123'])]
name='Rasoi' location='Rajasthan' foods=[Food(name='1st dish', price=1.0, ingredients=['123']), Food(name='2nd Dish', price=2.85, ingredients=None)]
{'name': 'Rasoi', 'location': 'Rajasthan', 'foods': [{'name': '1st dish', 'price': 1.0, 'ingredients': ['123']}, {'name': '2nd Dish', 'price': 2.85, 'ingredients': None}]}


In [27]:
from typing import Optional
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr

class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner: Owner
    address: Address
    employees: str = "Fruth"
    number_of_seats: PositiveInt
    delivery: bool
    website: HttpUrl
    
    # Creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "john.doe@example.com"
    },
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    number_of_seats=50,
    delivery=True,
    website="http://tastybites.com"
)

print(restaurant_instance)


name='Tasty Bites' owner=Owner(name='John Doe', email='john.doe@example.com') address=Address(street='123, Flavor Street', city='Tastytown', state='TS', zip_code='12345') employees='Fruth' number_of_seats=50 delivery=True website=HttpUrl('http://tastybites.com/')


Field Validators

In [32]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.title()

try:
    owner_instance = Owner(name="John Doe", email="john.doe@example.com")
    print(owner_instance)
except ValueError as e:
    print(e)

name='John Doe' email='john.doe@example.com'


Model Validator

In [35]:
from typing import Any
from pydantic import BaseModel, EmailStr, ValidationError, model_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @model_validator(mode='before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError('card_number should not be included')
        return data
    
    @model_validator(mode='after')
    def check_name_contains_space(self) -> 'Owner':
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self



print(Owner(name="John Doe", email="john.doe@example.com")) 

try:
    # Owner(name="JohnDoe", email="john.doe@example.com", password="password123")
    Owner(name="John Doe", email="john.doe@example.com")
except ValidationError as e:
    print(e) 


name='John Doe' email='john.doe@example.com'


Field different use-cases

In [41]:
from uuid import uuid4
from pydantic import BaseModel, Field

class User(BaseModel):
    id: int = Field(default_factory= lambda: uuid4().hex)

user= User()
print(user)

id='537fff7059a54c73962c14275c63a840'


In [45]:
class User(BaseModel):
    name: str = Field(..., alias= 'username')

user = User(username = 'johndoe')
print(user)
print(user.model_dump(by_alias = True))
print(user.model_dump(by_alias = False))

name='johndoe'
{'username': 'johndoe'}
{'name': 'johndoe'}
